In [1]:
import torch
import torch_directml
import pandas as pd
import time

gpu1 = torch_directml.device(1)

In [2]:
import torch.multiprocessing as multiprocessing
from torch.multiprocessing import Pool, Process, set_start_method
# try:
#     set_start_method('spawn')
# except RuntimeError:
#     pass
manager = multiprocessing.Manager()
shared_dict = manager.dict({'init_var': 0})
shared_temp_number = manager.Value('i', 0)
lock = manager.Lock()

In [3]:
# Import data of field and epitaxy
path = '../lib/data/HXDQW_Ey230602.csv'
df = pd.read_csv(path)
coor = df['z'] # unit: m, resolution: 0.1 nm
index = df['index_x']
Ey_real = df['mode4_Ey(real)']
Ey_imag = df['mode4_Ey(imag)']
coor = torch.Tensor(coor.to_numpy()).to(gpu1)
index = torch.Tensor(index.to_numpy()).to(gpu1)
Ey_real = torch.Tensor(Ey_real.to_numpy()).to(gpu1)
Ey_imag = torch.Tensor(Ey_imag.to_numpy()).to(gpu1)
Ey = torch.stack((Ey_real, Ey_imag), dim=1)
Ey

tensor([[ 0.0000e+00,  0.0000e+00],
        [-1.7497e-06, -3.5657e-06],
        [-3.3209e-06, -6.2084e-06],
        ...,
        [ 1.6124e-09,  2.8240e-09],
        [ 8.5337e-10,  1.6240e-09],
        [ 0.0000e+00,  0.0000e+00]], device='privateuseone:1')

In [4]:
def complex_tensor_integral(tensor, y):
    '''
    tensor: tensor with two columns, the first column is real part, the second column is imaginary part
    y: real tensor
    '''
    real = tensor[:, 0]
    imag = tensor[:, 1]
    real_integral = torch.trapz(real, y)
    imag_integral = torch.trapz(imag, y)
    return torch.stack((real_integral, imag_integral), dim=0)

In [5]:
i = 0
start_time = time.time()
while i < 10000:
    complex_tensor_integral(Ey, coor)
    i += 1
end_time = time.time()
print(end_time - start_time)

2.058516263961792


In [6]:
import numpy as np
def complex_numpy_integral(array, y):
    '''
    array: array with two columns, the first column is real part, the second column is imaginary part
    y: real array
    '''
    real = array[:, 0]
    imag = array[:, 1]
    real_integral = np.trapz(real, y)
    imag_integral = np.trapz(imag, y)
    return np.array([real_integral, imag_integral])

In [7]:
array = Ey.cpu().numpy()
coor_array = coor.cpu().numpy()

In [8]:
i = 0
start_time = time.time()
while i < 10000:
    complex_numpy_integral(array, coor_array)
    i += 1
end_time = time.time()
print(end_time - start_time)

0.6997151374816895


In [9]:
# After Testing, we find that the speed of numpy is faster than torch with gpu.
# So this script is not used in the project.